In [1]:
import collections
import itertools

# Our numerical workhorses
import numpy as np
import pandas as pd
import scipy.optimize
import scipy.stats as st

# Numerical differentiation packages
import numdifftools as ndt

# Our main MCMC package
import emcee

# Import pyplot for plotting
import matplotlib.pyplot as plt

# Seaborn, useful for graphics
import seaborn as sns

# Corner is useful for displaying MCMC results
import corner

# Magic function to make matplotlib inline; other style specs must come AFTER
%matplotlib inline

# This enables high res graphics inline (only use with static plots (non-Bokeh))
# SVG is preferred, but there is a bug in Jupyter with vertical lines
%config InlineBackend.figure_formats = {'png', 'retina'}

# JB's favorite Seaborn settings for notebooks
rc = {'lines.linewidth': 2, 
      'axes.labelsize': 18, 
      'axes.titlesize': 18, 
      'axes.facecolor': 'DFDFE5'}
sns.set_context('notebook', rc=rc)
sns.set_style('darkgrid', rc=rc)

## 1A
Reversal in worms

In [2]:
# Load DataFrame
df = pd.read_csv('data/bi1x_2015_c_elegans_optogenetics.csv',
                comment='#')
df = pd.melt(df, id_vars=['Group', 'Worm'], value_vars=['WT', 'AVA', 'ASH'], 
             value_name='reversal', var_name='strain')


In [3]:
def revs_trials(df, strain):
    """
    Return number of reversals and number of trials.
    """
    inds = (df['strain'] == strain) & (df['reversal'] >= 0)
    n_r = df[inds]['reversal'].sum()
    n = df[inds]['reversal'].count()
    
    return n_r, n

def log_posterior(p, n_r, n):
    """
    Log posterior of reversal measurements.
    """

    # Zero probability of having p < 0 or p > 1
    if p < 0 or p > 1:
        return -np.inf
    
    return st.nbinom.logpmf(n_r, n, p).sum()




    


In [48]:
n_dim = 1        # number of parameters in the model (just p)
n_walkers = 50   # number of MCMC walkers
n_burn = 1000     # "burn-in" period to let chains stabilize
n_steps = 5000   # number of MCMC steps to take after burn-in


n_r, n = revs_trials(df, 'AVA')
p0 = np.empty((n_walkers, n_dim))
p0[:,0] = np.random.uniform(0,1, n_walkers)             # AVA
#p0[:,1] = np.random.uniform(0, 1, n_walkers)             # ASH

sampler = emcee.EnsembleSampler(n_walkers, n_dim, log_posterior, 
                                args=(n,n_r), threads=4)

# Do burn-in
pos, prob, state = sampler.run_mcmc(p0, n_burn, storechain=False)

KeyError: 'strain'

In [38]:
#actually do MCMC
# Sample again, starting from end burn-in state
_ = sampler.run_mcmc(pos, n_steps)

In [41]:

# Get the index of the most probable parameter
max_ind = np.argmax(sampler.flatlnprobability)
#max_ind

# Pull out values.
AVA_MAP = sampler.flatchain[max_ind]
AVA_MAP
# Print the results
#print("""
#Most probable parameter value:
#p: {1:.1f}
#""".format(p_MAP))

array([ 0.4782591])

In [42]:
n_dim = 1        # number of parameters in the model (n_r and p)
n_walkers = 50   # number of MCMC walkers
n_burn = 1000     # "burn-in" period to let chains stabilize
n_steps = 5000   # number of MCMC steps to take after burn-in

n_r, n = revs_trials(df, 'ASH')
p0 = np.empty((n_walkers, n_dim))


p0[:,0] = np.random.uniform(0,1, n_walkers)             # ASH


sampler = emcee.EnsembleSampler(n_walkers, n_dim, log_posterior, 
                                 args=(n,n_r), threads=4)

# Do burn-in
pos, prob, state = sampler.run_mcmc(p0, n_burn, storechain=False)

In [43]:
#actually do MCMC
# Sample again, starting from end burn-in state
_ = sampler.run_mcmc(pos, n_steps)

In [44]:
# Get the index of the most probable parameter
max_ind = np.argmax(sampler.flatlnprobability)
#max_ind

# Pull out values.
ASH_MAP = sampler.flatchain[max_ind]
ASH_MAP

array([ 0.20454558])

In [45]:
AVA_MAP-ASH_MAP

array([ 0.27371351])

## 1B
Dorsal gradients in wt vs venus fusion

In [46]:
# Load data
df = pd.read_csv('data/reeves_gradient_width_various_methods.csv', comment='#',header=[0,1])

# Check it out
df.head()

wt                dl1/+; dl-venus/+                          \
  wholemounts cross-sections       anti-Dorsal anti-Venus Venus (live)   
0      0.1288         0.1327            0.1482     0.1632       0.1666   
1      0.1554         0.1457            0.1503     0.1671       0.1753   
2      0.1306         0.1447            0.1577     0.1704       0.1705   
3      0.1413         0.1282            0.1711     0.1779          NaN   
4      0.1557         0.1487            0.1342     0.1483          NaN   

  dl1/+; dl-gfp/+                      
      anti-Dorsal anti-GFP GFP (live)  
0          0.2248   0.2389     0.2412  
1          0.1891   0.2035     0.1942  
2          0.1705   0.1943     0.2186  
3          0.1735   0.2000     0.2104  
4          0.2135   0.2560     0.2463

In [47]:
crosssection= df['wt']['cross-sections'].dropna()
venus_dorsal= df['dl1/+; dl-venus/+']['anti-Dorsal'].dropna()

In [60]:
def peak(p, x):
    """
    Theroetical peak (Gaussian + bg)
    """
    a, b, mu, sigma = p
    return a + b * np.exp(-(x - mu)**2 / 2 / sigma**2)


def resid(p, x, f):
    """
    Residuals for peak fitting.
    """
    return f - peak(p, x)


def log_post(p, x, f):
    """
    Log of the posterior.
    """
    return -len(x) / 2 * np.log(np.sum(resid(p, x, f)**2))


In [62]:
n_dim = 2        # number of parameters in the model (just p)
n_walkers = 50   # number of MCMC walkers
n_burn = 1000     # "burn-in" period to let chains stabilize
n_steps = 5000   # number of MCMC steps to take after burn-in


p0 = np.empty((n_walkers, n_dim))
p0[:,0] = np.random.uniform(0,1, n_walkers)             # wt
p0[:,1] = np.random.uniform(0, 1, n_walkers)             # venus

sampler = emcee.EnsembleSampler(n_walkers, n_dim, log_posterior, 
                                args=(df['wt']['cross-sections'].dropna(),df['dl1/+; dl-venus/+']['anti-Dorsal'].dropna()), threads=4)

# Do burn-in
pos, prob, state = sampler.run_mcmc(p0, n_burn, storechain=False)

emcee: Exception while calling your likelihood function:
emcee: Exception while calling your likelihood function:
emcee: Exception while calling your likelihood function:
emcee: Exception while calling your likelihood function:
  params: [ 0.19891536  0.64668545]
  params: [ 0.07036813  0.43773097]
  params: [ 0.42729021  0.20997308]
  params: [ 0.73902889  0.67207817]
  args: (0      0.1327
1      0.1457
2      0.1447
3      0.1282
4      0.1487
5      0.1203
6      0.1315
7      0.1463
8      0.1458
9      0.1402
10     0.1330
11     0.1375
12     0.1576
13     0.1498
14     0.1532
15     0.1352
16     0.1346
17     0.1364
18     0.1430
19     0.1440
20     0.1805
21     0.1688
22     0.1532
23     0.1681
24     0.1442
25     0.1362
26     0.1391
27     0.1606
28     0.1742
29     0.1335
        ...  
122    0.1443
123    0.1329
124    0.1567
125    0.1388
126    0.1449
127    0.1548
128    0.1727
129    0.1420
130    0.1735
131    0.1641
132    0.1756
133    0.1691
134    0.1599
135

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "//anaconda/lib/python3.4/site-packages/emcee/ensemble.py", line 505, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "//anaconda/lib/python3.4/site-packages/emcee/ensemble.py", line 505, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "//anaconda/lib/python3.4/site-packages/emcee/ensemble.py", line 505, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "//anaconda/lib/python3.4/site-packages/emcee/ensemble.py", line 505, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "<ipython-input-3-ed3efa986d27>", line 17, in log_posterior
    if p < 0 or p > 1:
  File "<ipython-input-3-ed3efa986d27>", line 17, in log_posterior
    if p < 0 or p > 1:
  File "<ipython-input-3-ed3efa986d27>", line 17, in log_posterior
    if p < 0 or p > 1:
  File "<ipython-input-3-ed3efa986d27>", l

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()